# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np 
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.dtypes

CustomerID      int64
FirstName      object
LastName       object
SalesID         int64
ProductID       int64
ProductName    object
Quantity        int64
dtype: object

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
datagb = data.groupby(['CustomerID', 'ProductName']).sum()
datagb

SalesID  ProductID  Quantity
CustomerID ProductName                                                     
33         Apricots - Dried                     172592        324         1
           Assorted Desserts                   5363705        219         1
           Bandage - Flexible Neon             6299165        273         1
           Bar Mix - Pina Colada, 355 Ml       3147536        281         1
           Beans - Kidney, Canned              6507033        300         1
           Beef - Chuck, Boneless              3827834        439         1
           Beef - Prime Rib Aaa                5020972        286         1
           Beer - Original Organic Lager       1899667        432         1
           Beer - Rickards Red                 4280329        284         1
           Black Currants                       814477        110         1
           Blackberries                        6022221        441         1
           Bread - Italian Roll With Herbs     2214661        287         1
           Bread Foccacia Whole                2277923        378         1
           Cake - Box Window 10x10x2.5         6448281        383         1
           Cassis                              1888258         53         1
           Cheese - Cottage Cheese             1508062        333         1
           Crab - Imitation Flakes             6515659         23         1
           Fondant - Icing                     5056183        387         1
           French Pastry - Mini Chocolate      5638266        214         1
           General Purpose Trigger             2958865         91         1
           Grapes - Red                        4788871        267         1
           Grouper - Fresh                     1919415        490         2
           Hersey Shakes                       2768961        184         1
           Ice Cream Bar - Drumstick           1579966        154         1
           Lamb - Pieces, Diced                6509275        417         1
           Lettuce - Spring Mix                6562264        886         2
           Mangoes                             5869069        342         1
           Muffin Batt - Choc Chk              5150162        309         1
           Onions - Vidalia                     532838        322         1
           Pasta - Cheese / Spinach Bauletti   4226296        348         1
...                                                ...        ...       ...
98200      Onion Powder                        6005937        280        25
           Orange - Canned, Mandarin           4043235        288        25
           Pasta - Detalini, White, Fresh      1746414        257        25
           Puree - Passion Fruit               2378478        392        25
           Raspberries - Fresh                  714969         69        25
           Rice - Long Grain                    486150        359        25
           Rosemary - Dry                      1012242        173        25
           Rosemary - Primerba, Paste         11215217        204        50
           Rum - Coconut, Malibu               6698694        138        25
           Sage - Ground                       3932940         38        25
           Sausage - Liver                     1248631        344        25
           Sea Bass - Whole                   10743215        554        50
           Seedlings - Mix, Organic              49843        390        25
           Sole - Dover, Whole, Fresh           428968        222        25
           Soup - Campbells, Lentil            9863233        202        50
           Soup Campbells - Italian Wedding    3050876        119        25
           Soup Knorr Chili With Beans         3284766        256        25
           Soupfoamcont12oz 112con             5178755        263        25
           Spinach - Baby                      3134571         29        25
           Table Cloth 54x72 White              550966         36        25
           Tea - En

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
piv = pd.pivot_table(data,values = ['Quantity'], index = ['ProductName'],columns = ['CustomerID']).fillna(0)
# piv.apply(['CustomerID'])
piv

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mini Egg Roll, Shrimp      0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Mushroom Tart              0.0   0.0   0.0   0.0   0.0   0.0   
Appetizer - Sausage Rolls              0.0   0.0   0.0   0.0   0.0   0.0   
Apricots - Dried                       1.0   0.0   0.0   0.0   1.0   0.0   
Apricots - Halves                      0.0   0.0   1.0   0.0   0.0   0.0   
Apricots Fresh                         0.0   0.0   1.0   0.0   0.0   1.0   
Arizona - Green Tea                    0.0   0.0   0.0   0.0   0.0   0.0   
Artichokes - Jerusalem                 0.0   0.0   0.0   0.0   0.0   0.0   
Assorted Desserts                      1.0   0.0   0.0   0.0   0.0   0.0   
Bacardi Breezer - Tropical             0.0   1.0   1.0   0.0   0.0   0.0   
Bagel - Plain                          0.0   1.0   1.0   0.0   0.0   0.0   
Baking Powder                          0.0   0.0   0.0   1.0   0.0   0.0   
Banana - Leaves                        0.0   0.0   1.0   0.0   1.0   1.0   
Banana Turning                         0.0   0.0   0.0   0.0   0.0   1.0   
Bananas                                0.0   0.0   1.0   0.0   1.0   0.0   
Bandage - Fexible 1x3                  0.0   0.0   0.0   0.0   0.0   0.0   
Bandage - Flexible Neon                1.0   0.0   1.0   0.0   0.0   0.0   
Bar - Granola Trail Mix Fruit Nut      0.0   1.0   0.0   1.0   0.0   0.0   
Bar Mix - Pina Colada, 355 Ml          1.0   0.0   0.0   1.0   0.0   0.0   
Barramundi                             0.0   0.0   0.0   0.0   0.0   0.0   
Bay Leaf                               0.0   0.0   0.0   0.0   0.0   0.0   
Beans - Kidney White                   0.0   0.0   0.0   0.0   0.0   0.0   
Beans - Kidney, Canned                 1.0   0.0   0.0   0.0   0.0   0.0   
Beans - Kidney, Red Dry                0.0   0.0   0.0   0.0   0.0   0.0   
Beans - Wax                            0.0   0.0   0.0   0.0   1.0   0.0   
Beef - Chuck, Boneless                 1.0   0.0   0.0   0.0   0.0   0.0   
Beef - Ground Medium                   0.0   1.0   0.0   1.0   0.0   0.0   
Beef - Ground, Extra Lean, Fresh       0.0   0.0   0.0   0.0   0.0   1.0   
Beef - Inside Round                    0.0   0.0   0.0   0.0   0.0   0.0   
...                                    ...   ...   ...   ...   ...   ...   
Wine - Fume Blanc Fetzer               0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Gato Negro Cabernet             1.0   1.0   1.0   0.0   0.0   0.0   
Wine - Hardys Bankside Shiraz          0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Magnotta - Belpaese             0.0   0.0   0.0   1.0   0.0   0.0   
Wine - Magnotta - Cab Sauv             0.0   0.0   0.0   0.0   1.0   0.0   
Wine - Magnotta, Merlot Sr Vqa         0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Pinot Noir Latour               0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Prosecco Valdobiaddene          0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Red, Colio Cabernet             0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Red, Cooking                    0.0   0.0   0.0   1.0   0.0   0.0   
Wine - Red, Harrow Estates, Cab        0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Redchard Merritt                0.0   0.0   1.0   0.0   0.0   0.0   
Wine - Ruffino Chianti                 0.0   1.0   0.0   0.0   0.0   0.0   
Wine - Sogrape Mateus Rose             1.0   0.0   1.0   0.0   0.0   1.0   
Wine - Toasted Head                    0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Two Oceans Cabernet             1.0   0.0   0.0   1.0   1.0   0.0   
Wine - Valpolicella Masi               1.0   0.0   0.0   0.0   0.0   0.0   
Wine - Vidal Icewine Magnotta          0.0   0.0   0.0   0.0   0.0   0.0   
Wine - Vineland Estate Semi - Dry      0.0   0.0   0.0   0.0   0.0   0.0   
Wine -

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [11]:
def similarity_matrix(df):
    return pd.DataFrame(
        data = 1 / (1 + squareform(pdist(df.transpose()))),
        index = df.columns,
        columns = df.columns
    )

In [12]:
similarities = similarity_matrix(piv)

similarities.columns = similarities.columns.droplevel() 
# it just drops the level. ie kills that dam quantity that was there for no reason.

similarities_transp = similarities.transpose()

similarities_transp.columns = similarities_transp.columns.droplevel()
# it just drops the level. ie kills that dam quantity that was there for no reason.

similarities_transp

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
464,0.089695,0.083020,0.087047,0.086688,0.085638,1.000000,0.085638,0.092610,0.087779,0.087047,...,0.004814,0.005121,0.004993,0.005445,0.004920,0.005042,0.004877,0.005039,0.004556,0.004897
477,0.085297,0.084959,0.085638,0.085983,0.085638,0.085638,1.000000,0.090909,0.086333,0.087047,...,0.004816,0.005124,0.005005,0.005436,0.004928,0.005042,0.004882,0.005039,0.004556,0.004885
639,0.088913,0.083651,0.086333,0.091325,0.089301,0.092610,0.090909,1.000000,0.093498,0.091747,...,0.004814,0.005122,0.004993,0.005441,0.004917,0.005039,0.004880,0.005033,0.004563,0.004883
649,0.088152,0.085638,0.087047,0.085983,0.084959,0.087779,0.086333,0.093498,1.000000,0.088530,...,0.004811,0.005115,0.004999,0.005437,0.004923,0.005039,0.004886,0.005045,0.004558,0.004877


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [13]:
top_5 = similarities[33].sort_values(ascending = False).head(6)
list_top_6 = []
for n in top_5.index:
    list_top_6.append(n[1])
list_top_5 = list_top_6[1:6]
list_top_5
# large point here you neeed to remeber to account for the fact index is saved aas a touple(Gods I hate touples.)

[3909, 264, 3531, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [38]:
simmilars = data[data['CustomerID'].isin(list_top_5)]
simmilars

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
130,3909,Keri,Stafford,509197,270,Cheese Cloth No 100,1
159,264,Virgil,Small,24688,270,Cheese Cloth No 100,1
448,3909,Keri,Stafford,1834282,432,Beer - Original Organic Lager,1
887,3305,Bernard,O'Neill,2025094,102,"Rosemary - Primerba, Paste",1
928,3909,Keri,Stafford,4776765,417,"Lamb - Pieces, Diced",1
1038,2503,Kyle,Reeves,182355,417,"Lamb - Pieces, Diced",1
1233,3531,Ivan,Buck,4644989,17,"Wine - White, Mosel Gold",1
1253,3909,Keri,Stafford,2919014,159,Eggplant - Asian,1
1394,3531,Ivan,Buck,6165455,159,Eggplant - Asian,1
1660,2503,Kyle,Reeves,2031666,69,Raspberries - Fresh,1


In [45]:
# simmilars.groupby(['CustomerID','ProductName'])

In [46]:
# truesimilars = np.where(simmilars, data)

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [52]:
sim = simmilars.groupby(['ProductName'])['Quantity'].sum().sort_values(ascending=False)
sim

ProductName
Salsify, Organic                    4
Wine - Charddonnay Errazuriz        3
Wine - Ej Gallo Sierra Valley       3
Oranges - Navel, 72                 3
Quiche Assorted                     3
Bay Leaf                            3
Pecan Raisin - Tarts                3
Chocolate - Dark                    3
Towels - Paper / Kraft              2
Tofu - Firm                         2
Sauce - Rosee                       2
Sardines                            2
Pork - Hock And Feet Attached       2
Black Currants                      2
Sauerkraut                          2
Eggplant - Asian                    2
V8 - Berry Blend                    2
Chef Hat 20cm                       2
Beets - Candy Cane, Organic         2
Ice Cream Bar - Oreo Cone           2
Oil - Shortening - All - Purpose    2
Sausage - Liver                     2
Dried Figs                          2
Squid U5 - Thailand                 2
Bread - Rye                         2
Bread Crumbs - Panko                2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [ ]:
# need to check what the chosen number, 33 in this case, has not bought when compared to q6 dta list.
# 

In [50]:
q7data = data.groupby(['CustomerID', 'ProductName']).sum()

In [54]:
pd.merge(q7data, sim, on='ProductName' )

ValueError: can not merge DataFrame with instance of type <class 'pandas.core.series.Series'>

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [44]:
Records = {}

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.